# LangChain API Chain Integration Demo

This demo was orignially presented during the Austin LangChain User Group meetup on September 11th 2024.

The demo shows a simple LangGraph integration with a 3rd party API.

##Links:
presentation recording TODO

*   Presentation
*   [LangChain Interacting with APIs](https://python.langchain.com/v0.1/docs/use_cases/apis/)
*   [LangChain APIChain (deprecated)](https://api.python.langchain.com/en/latest/chains/langchain.chains.api.base.APIChain.html)
*   [LangSmith Trace](https://smith.langchain.com/public/86a0ceea-3eae-42ed-91cf-acc6ff9ffa5e/r)


##Setup

In [11]:
%pip install --upgrade --quiet  pygithub langchain langchain-community langchain-openai langgraph

In [12]:
import os
import requests
from google.colab import userdata
from langchain import PromptTemplate
from langchain_openai import ChatOpenAI
from typing import Annotated, Sequence, List, Dict
from typing_extensions import TypedDict

from langchain.chains.api.prompt import API_URL_PROMPT
from langchain_community.agent_toolkits.openapi.toolkit import RequestsToolkit
from langchain_community.utilities.requests import TextRequestsWrapper
from langchain_core.messages import BaseMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableConfig
from langgraph.graph import END, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt.tool_node import ToolNode

###Required Keys and environment variables

Keys are stored in Colab Secrets for this demo and called using `userdata.get()`

###Set the environment variables

In [13]:
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "atx_apichain_demo"

open_api_key = os.environ.get("OPEN_API_KEY")
langchain_api_key = os.environ.get("LANGCHAIN_API_KEY")


###Prepare the API Spec file  

In [14]:
api_spec = """
openapi: 3.0.0

info:
  version: 1.0.0
  title: xkcd
  description: 'A webcomic of romance, sarcasm, math, and language.'

servers:
  - url: https://xkcd.com/
    description: Official xkcd JSON interface

paths:
  # Retrieve the current comic
  /info.0.json:
    get:
      # A list of tags to logical group operations by resources and any other
      # qualifier.
      tags:
        - comic
      description: Returns comic based on ID
      summary: Find latest comic
      # Unique identifier for the operation, tools and libraries may use the
      # operationId to uniquely identify an operation.
      operationId: getComic
      responses:
        '200':
          description: Successfully returned a comic
          content:
            application/json:
              schema:
                # Relative reference to prevent duplicate schema definition.
                $ref: '#/components/schemas/Comic'
  # Retrieve a comic by ID
  /{id}/info.0.json:
    get:
      tags:
        - comic
      description: Returns comic based on ID
      summary: Find comic by ID
      operationId: getComicById
      parameters:
        - name: id
          in: path
          required: true
          schema:
            type: integer
      responses:
        '200':
          description: Successfully returned a commmic
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/Comic'

components:
  schemas:
    Comic:
      type: object
      properties:
        month:
          type: string
        num:
          type: integer
        link:
          type: string
        year:
          type: string
        news:
          type: string
        safe_title:
          type: string
        transcript:
          type: string
        alt:
          type: string
        img:
          type: string
        title:
          type: string
        day:
          type: string
"""

###Setup the LLM and LangChain toolkit
Here we are using OpenAI's GPT-4o-mini model as recommended by the LangChain team.

In [15]:
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
toolkit = RequestsToolkit(
    requests_wrapper=TextRequestsWrapper(headers={}),
    allow_dangerous_requests=True,
    open_api_key=open_api_key
)

tools = toolkit.get_tools()

###Setup LangChain Chain
Notice the use of `API_URL_PROMPT` which is a template prompt built into [LangChain tools](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/api/prompt.py) to help you with constructing API URLs.


In [16]:
api_request_chain = (
    API_URL_PROMPT.partial(api_docs=api_spec)
    | llm.bind_tools(tools, tool_choice="any")
)

### Create Chain State


In [17]:
class ChainState(TypedDict):
    """LangGraph state."""
    messages: Annotated[Sequence[BaseMessage], add_messages]

### Graph Nodes

In [18]:
async def acall_request_chain(state: ChainState, config: RunnableConfig):
    last_message = state["messages"][-1]
    response = await api_request_chain.ainvoke(
        {"question": last_message.content}, config
    )
    return {"messages": [response]}

async def acall_model(state: ChainState, config: RunnableConfig):
    response = await llm.ainvoke(state["messages"], config)
    return {"messages": [response]}

### Build the Graph

In [19]:
graph_builder = StateGraph(ChainState)
graph_builder.add_node("call_tool", acall_request_chain)
graph_builder.add_node("execute_tool", ToolNode(tools))
graph_builder.add_node("call_model", acall_model)
graph_builder.set_entry_point("call_tool")
graph_builder.add_edge("call_tool", "execute_tool")
graph_builder.add_edge("execute_tool", "call_model")
graph_builder.add_edge("call_model", END)
chain = graph_builder.compile()

### Run the Chain

In [20]:
example_query = "what is the title, date created and direct link of the latest XKCD comic?"
#example_query = "what date was the 1000th comic released?"

events = chain.astream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
async for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

what is the title, date created and direct link of the latest XKCD comic?
================================== Ai Message ==================================
Tool Calls:
  requests_get (call_GvLfiEe1qPGdE02YGmo2nyFY)
 Call ID: call_GvLfiEe1qPGdE02YGmo2nyFY
  Args:
    url: https://xkcd.com/info.0.json
================================= Tool Message =================================
Name: requests_get

{"month": "9", "num": 2983, "link": "", "year": "2024", "news": "", "safe_title": "Monocaster", "transcript": "", "alt": "My competitors say the tiny single tiny caster is unsafe, unstable, and offers no advantages over traditional designs, to which I say: wow, why are you guys so mean? I thought we were friends!", "img": "https://imgs.xkcd.com/comics/monocaster.png", "title": "Monocaster", "day": "9"}
================================== Ai Message ==================================

The latest XKCD comic is titl

## Integrate with Search

now we want to expand on the limited API calls from XKCD to improve usability by utilizing GPT-4o to curate a list of popular comics with the option to filter on topic or genera.





### Add LLM query

In [21]:
import re

def parse_ai_response(response_content: str) -> List[int]:
    # Extract the part that looks like a Python list
    match = re.search(r'\[[\s\S]*?\]', response_content)
    if not match:
        raise ValueError("Could not find a valid list in the AI response")

    list_str = match.group(0)

    # Extract numbers from the list
    numbers = re.findall(r'\d+', list_str)

    # Convert to integers
    return [int(num) for num in numbers]


def curate_popular_xkcd_comics(num_comics: int = 5, catagory: str = "") -> List[Dict]:
    llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.2)

    prompt = f"""As an AI with knowledge about XKCD comics, please provide an ordered list of the top {num_comics} most popular {catagory} XKCD comics.
    Include only the comic numbers in a Python list format. For example:

    top_xkcd_comics = [303, 327, 353]
    """

    response = llm.invoke(prompt)

    # Parse the response to extract the comic numbers
    return parse_ai_response(response.content)


In [22]:
resp = curate_popular_xkcd_comics(5, "Programming")
print(resp)

[353, 303, 927, 1957, 1340]


### Update Chain State with query results

In [23]:
class ChainState(TypedDict):
    """LangGraph state."""
    messages: Annotated[Sequence[BaseMessage], add_messages]
    popular_comics: List[int]

### Add Nodes

Add node executable `curate_comics` and update `acall_model` with LLM results.

In [24]:
async def curate_comics(state: ChainState, config: RunnableConfig):
    popular_comics = curate_popular_xkcd_comics()
    return {"popular_comics": popular_comics}

In [25]:
from langchain.schema import SystemMessage

async def acall_model(state: ChainState, config: RunnableConfig):
    messages = state["messages"]
    popular_comics = state["popular_comics"]
    response = await llm.ainvoke(messages, config)
    return {"messages": [response]}

Update `acall_request_chain` to use the output list from `curate_comics`

In [26]:
async def acall_request_chain(state: ChainState, config: RunnableConfig):
    last_message = state["popular_comics"]
    response = await api_request_chain.ainvoke(
        {"question": last_message}, config
    )
    return {"messages": [response]}

### Update Graph

In [27]:
graph_builder = StateGraph(ChainState)
graph_builder.add_node("curate_comics", curate_comics)
graph_builder.add_node("call_tool", acall_request_chain)
graph_builder.add_node("execute_tool", ToolNode(tools))
graph_builder.add_node("call_model", acall_model)
graph_builder.set_entry_point("curate_comics")
graph_builder.add_edge("curate_comics", "call_tool")
graph_builder.add_edge("call_tool", "execute_tool")
graph_builder.add_edge("execute_tool", "call_model")
graph_builder.add_edge("call_model", END)
chain = graph_builder.compile()

# Query

In [28]:
example_query = "provide a list of the top 5 most popular XKCD comics about timetravel. Include a link to each and the date they were created?"

events = chain.astream(
    {"messages": [("user", example_query)], "popular_comics": []},
    stream_mode="values",
)

async for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

provide a list of the top 5 most popular XKCD comics about timetravel. Include a link to each and the date they were created?
================================ Human Message =================================

provide a list of the top 5 most popular XKCD comics about timetravel. Include a link to each and the date they were created?
================================== Ai Message ==================================
Tool Calls:
  requests_get (call_AdkmKPw9XduiprawHU0McDN0)
 Call ID: call_AdkmKPw9XduiprawHU0McDN0
  Args:
    url: https://xkcd.com/353/info.0.json
  requests_get (call_gJSFU4guGEhzuxNWoo1YmBOo)
 Call ID: call_gJSFU4guGEhzuxNWoo1YmBOo
  Args:
    url: https://xkcd.com/303/info.0.json
  requests_get (call_r7nSoHAQGHN5kKoRtUYEURI0)
 Call ID: call_r7nSoHAQGHN5kKoRtUYEURI0
  Args:
    url: https://xkcd.com/327/info.0.json
  requests_get (call_a0wHTjWYDTePuVnCkW0lLxYM)
 Call ID: call_a0wHTjWYDTePuVnCkW